In [1]:
import torch
import torch.utils.data as data
import pandas as pd

In [2]:
from data import DigitsDateset
from models.DigitsRecgnitionNetwork import DigitsRecognizer, DigitsPredictor
from utils import multi_target_loss, sum_list_strs, show

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
ds = DigitsDateset('train')
net = DigitsRecognizer()
net.to(device)
dl = data.DataLoader(dataset=ds, batch_size=16, shuffle=True)

In [9]:
net.train()
optim = torch.optim.Adam(net.parameters(), lr=0.0001)
for _ in range(5):
    mean_loss = 0
    for i, (X, y) in enumerate(dl):
        optim.zero_grad()
        X, y = X.to(device), y.to(device)
        loss = multi_target_loss(y, net(X))
        loss.backward()
        optim.step()
        mean_loss += loss.item()
        if (i + 1) % 10 == 0:
            print(mean_loss / 10)
            mean_loss = 0

0.194980039447546
0.21359469145536422
0.22774698138236998
0.19150238260626792
0.19875237792730333
0.16399948969483374
0.1488884322345257
0.15746908709406854
0.12732644081115724
0.13592083975672722
0.15191973820328714
0.1416444804519415
0.1028466872870922
0.10573423467576504
0.1124427456408739
0.1113104123622179
0.11707773581147193
0.10519921220839024
0.0832416620105505
0.09673750475049019
0.10055862702429294
0.08058158867061138
0.06822032313793898
0.10273192413151264
0.07312489487230778
0.06449894569814205
0.07920324243605137
0.07456098031252623
0.07319563198834658
0.0830477885901928


In [10]:
# test accuracy
# TODO implement it better
with torch.no_grad():
    net.eval()
    net.to(device)
    total_acc = 0
    for X, y in dl:
        X, y = X.to(device), y.to(device)
        logits = net(X)
        total_acc += (logits.argmax(dim=2) == y).all(dim=1).sum()
    print(total_acc / 1000)

tensor(0.9490, device='cuda:0')


In [11]:
with torch.no_grad():
    net.eval()
    net.to(device)
    test_ds = DigitsDateset('test')
    test_dl = data.DataLoader(test_ds, batch_size=16, shuffle=False)
    predictions = []
    for X in test_dl:
        X = X.to(device)
        logits = net(X)
        predictions += DigitsPredictor()(logits)

    # make submission file
    submission = pd.DataFrame(columns=['filename', 'result'])
    submission['filename'] = [f'test_{i + 1}.jpg' for i in range(500)]
    # TODO: add comments here
    submission['result'] = [sum_list_strs(list_strs)[0: 5] for list_strs in predictions]
    submission.to_csv(f'./submission.csv', header=True, index=False)

In [12]:
submission['result'].sample(10)

412    00466
97     00584
489    00247
269    00007
377    00130
323    00108
37     00198
204    00038
378    00041
34     00475
Name: result, dtype: object